# Technical Report
### NYC Homeless Shelter Population Forecasting

---

### Problem Statement
In this project, I forecasted the total number of individuals sheltered by the NYC Department of Homeless Services (DHS). I used DHS Daily Report publicly available from [NYC Open Data](https://opendata.cityofnewyork.us) to build Time Series model. I aimed to help understand homeless communities better by examining different factors that change the number of indivuals in DHS shelter and to help better prepare for those who are in need of a space at a shelter by forecasting the general demand.

### Data Collection
1. DHS Daily Report (Source: https://data.cityofnewyork.us/Social-Services/DHS-Daily-Report/k46n-sa2m)

DHS Daily Report was scraped using [Socrata Open Data API (SODA)](https://dev.socrata.com/foundry/data.cityofnewyork.us/k46n-sa2m). The report includes the number of total individuals in Shelter and the breakdown of the shelter population such as the total number of children in shelter from 2013. This report is updated daily and the update is availble through the API. Although the data did not have any null values, there were some missing dates indicating the report was not made on some days. For this reason, the daily data was resampled into weekly average data for analysis.

### Database (PostgreSQL)

### Exploratory Data Analysis

### Modeling

### Data Visualization

### Next Steps